In [55]:
globals().clear()


from __future__ import (absolute_import, division, print_function, unicode_literals)
from builtins import * 
from io import BytesIO, TextIOWrapper
from zipfile import ZipFile
import urllib.request
import csv
from shapely.geometry import Point
import geopandas as gpd

import time
import numpy as np
import zarr
from shapely.geometry import Polygon, Point
import pandas as pd
import geopandas as gpd
import s3fs
import os


os.chdir('C:/Users/lbeatty/Documents/Lauren_MIP_Contribution/')


In [56]:
def rect(i, w, s, e, n):
    x = [w[i], e[i], e[i], w[i], w[i]]
    y = [s[i], s[i], n[i], n[i], s[i]]
    return x, y

def poly(sr):
    ret = []
    w = sr["W"][:]
    s = sr["S"][:]
    e = sr["E"][:]
    n = sr["N"][:]
    for i in range(52411):
        x, y = rect(i, w, s, e, n)
        ret.append(Polygon([[x[0],y[0]],[x[1],y[1]],[x[2],y[2]],
                            [x[3],y[3]],[x[4],y[4]]]))
    return ret

In [57]:
rl = 's3://inmap-model/isrm_v1.2.1.zarr/'
url = 's3://inmap-model/isrm_v1.2.1.zarr/'

fs = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-east-2'))
sr = zarr.open(s3fs.S3Map(url, s3=fs, check=False), mode="r")

p = poly(sr)


In [58]:
scenario = '26z-short-base-200'
model = 'GenX'

year='2030'
emis = gpd.read_file('InMap/MIP_Emissions/marginal_emissions_'+year+'.shp')
fact = 28766.639

In [59]:
url = 's3://inmap-model/isrm_v1.2.1.zarr/'
fs = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-east-2'))
sr = zarr.open(s3fs.S3Map(url, s3=fs, check=False), mode="r")

df = pd.DataFrame({'Location': range(52411)})
p = poly(sr)
gdf = gpd.GeoDataFrame(df, geometry=p)

# join the emis dataframe into the grid dataframe
emis.crs = "+proj=longlat"
gdf.crs = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"
emis = emis.to_crs(gdf.crs)
join_right_df = gdf.sjoin(emis, how="right")
print("Finished joining the dataframes.")

Finished joining the dataframes.


In [60]:
index = join_right_df.Location.tolist()

ppl = np.unique(join_right_df.Location.tolist())

num = range(0,len(ppl))

dictionary = dict(zip(ppl, num))

SOA = sr['SOA'].get_orthogonal_selection(([0], ppl, slice(None)))
print("SOA data is allocated.")
pNO3 = sr['pNO3'].get_orthogonal_selection(([0], ppl, slice(None)))
print("pNO3 data is allocated.")
pNH4 = sr['pNH4'].get_orthogonal_selection(([0], ppl, slice(None)))
print("pNH4 data is allocated.")
pSO4 = sr['pSO4'].get_orthogonal_selection(([0], ppl, slice(None)))
print("pSO4 data is allocated.")
PM25 = sr['PrimaryPM25'].get_orthogonal_selection(([0], ppl, slice(None)))
print("PrimaryPM25 data is allocated.")

<bound method NDFrame.head of         index_left  Location  Longitude  Latitude           NOx           SOx  \
0              841       841  -124.2103  40.74150  5.844717e-07  3.021844e-08   
1              841       841  -124.2103  40.74150  1.912806e-06  2.770003e-08   
2              841       841  -124.2103  40.74150  9.549104e-04  2.404413e-05   
3              841       841  -124.2103  40.74150  5.844717e-07  3.021844e-08   
4              841       841  -124.2103  40.74150  6.579902e-07  3.401950e-08   
...            ...       ...        ...       ...           ...           ...   
136019       51802     51802   -67.9944  45.48220  0.000000e+00  0.000000e+00   
136020       51802     51802   -67.9710  45.61498  0.000000e+00  0.000000e+00   
136021       51707     51707   -67.8111  46.54422  0.000000e+00  0.000000e+00   
136022       51885     51885   -67.5477  44.67810  5.412812e-05  4.669999e-05   
136023       51886     51886   -67.4012  45.15540  0.000000e+00  0.000000e+00  

In [ ]:
exposure_data = pd.DataFrame()
BlackPop = sr['Black'][0:52411]
AsianPop = sr['Asian'][0:52411]
NativePop = sr['Native'][0:52411]
LatinoPop = sr['Latino'][0:52411]
WhiteNoLatPop = sr['WhiteNoLat'][0:52411]

pollution_dict = dict(zip(['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5'], [SOA, pNO3, pNH4, pSO4, PM25]))

group=1
for column in [BlackPop, AsianPop, NativePop, LatinoPop, WhiteNoLatPop]:
    sumpop = sum(column)
    for pollutant in ['VOC', 'NOx', 'NH3', 'SOx', 'PM2_5']:
            for i in range(len(index)):
                exposure = pollution_dict[pollutant][0, dictionary[index[i]], :]*emis[pollutant][i]*fact
                weighted_exposure = sum(exposure*column)/sumpop

                data = {'Group':[group], 'Pollutant':pollutant, 'Exposure': [weighted_exposure], 'Cluster': [emis.Resource[i]]}
                df = pd.DataFrame(data)
                exposure_data = pd.concat([exposure_data, df])
                if i % 5000 == 0:
                    print("Iteration:", i)
    group=group+1
    

Iteration: 0
Iteration: 5000


KeyboardInterrupt: 

In [ ]:
exposure_data.loc[exposure_data['Group'] == 1, 'Race'] = 'Black'
exposure_data.loc[exposure_data['Group'] == 2, 'Race'] = 'Asian'
exposure_data.loc[exposure_data['Group'] == 3, 'Race'] = 'Native'
exposure_data.loc[exposure_data['Group'] == 4, 'Race'] = 'Latino'
exposure_data.loc[exposure_data['Group'] == 5, 'Race'] = 'WhiteNoLat'

exposure_data_collapsed = exposure_data.groupby(['Race', 'Cluster','Pollutant']).agg({'Exposure':'sum'}).reset_index()